In [ ]:
import mne
from config_import import get_events, event_id

%matplotlib qt

In [ ]:
raw_fname = 'pogs_meg/ll_180197/220217/run1_raw.fif'
raw = mne.io.read_raw(raw_fname, allow_maxshield="yes")

In [ ]:
bads = ['MEG1613', 'MEG0213', 'MEG0311', 'MEG0813', 'MEG1542', 'MEG1541',
        'MEG1811', 'MEG0623', 'MEG0241']
raw.info['bads'] = bads

In [ ]:
cal_fname = "meg_conf/sss_cal_nsp_2017.dat"
ct_fname = "meg_conf/ct_sparse_nsp_2017.fif"
raw = mne.preprocessing.maxwell_filter(
    raw, origin="auto", calibration=cal_fname, cross_talk=ct_fname)

In [ ]:
l_freq, h_freq = 0.1, 40
raw.filter(l_freq, h_freq)

In [ ]:
events = get_events(raw)
epochs = mne.Epochs(raw, events, event_id, tmin=-0.1, tmax=1., baseline=None)

In [ ]:
evoked = epochs["reference"].average()
evoked.plot_joint()

# Now let's look at the MRI data

In [ ]:
subjects_dir = "bids_data/derivatives/freesurfer/subjects"
subject = "sub-09"

In [ ]:
mne.viz.plot_bem(subject=subject, subjects_dir=subjects_dir,
                 orientation='coronal');

In [ ]:
trans = mne.read_trans('pogs_meg/ll_180197/220217/sub-09-trans.fif')

In [ ]:
fig = mne.viz.plot_alignment(evoked.info, trans, subject=subject, dig=True,
                             subjects_dir=subjects_dir, verbose=True);

In [ ]:
mne.gui.coregistration(subject=subject, subjects_dir=subjects_dir, inst=raw_fname)

# Now let's do some source imaging

In [ ]:
raw_er = mne.io.read_raw_fif('pogs_meg/ll_180197/220217/emptyroom.fif', allow_maxshield="yes")
raw_er.info["bads"] = bads
raw_er.info["dev_head_t"] = raw.info["dev_head_t"]
raw_er.set_montage(raw.get_montage())
raw_er = mne.preprocessing.maxwell_filter(
    raw_er, origin="auto", calibration=cal_fname, cross_talk=ct_fname,
)
raw_er.filter(l_freq, h_freq)

In [ ]:
raw_er.plot_psd(fmax=60);

In [ ]:
cov = mne.compute_raw_covariance(raw_er)

In [ ]:
mne.viz.plot_cov(cov, info=raw_er.info)

In [ ]:
evoked.plot(noise_cov=cov);

In [ ]:
evoked.plot_white(noise_cov=cov);

In [ ]:
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject=subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

In [ ]:
src = mne.setup_source_space(subject, spacing='oct6',
                             subjects_dir=subjects_dir,
                             add_dist=False)

In [ ]:
fwd = mne.make_forward_solution(raw_fname, trans=trans,
                                src=src, bem=bem,
                                meg=True, # include MEG channels
                                eeg=False, # include EEG channels
                                mindist=5.0, # ignore sources <= 5mm from inner skull
                                n_jobs=1) # number of jobs to run in parallel

In [ ]:
from mne.minimum_norm import make_inverse_operator, apply_inverse

info = evoked.info
inverse_operator = make_inverse_operator(info, fwd, cov,
                                         loose=0.2, depth=0.8)

In [ ]:
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2
stc = apply_inverse(evoked, inverse_operator, lambda2,
                    method=method, pick_ori=None)
stc

In [ ]:
brain = stc.plot(surface='inflated', hemi='rh', subjects_dir=subjects_dir)